In [1]:
using Pkg
using DrWatson
@quickactivate

using DataFrames
using Mill
using JsonGrinder
# using PyCall
using CSV
using Dates
using Statistics
using Missings

In [2]:
# TODO: add hour, minute to the time features extracted to Mill

In [3]:
cards = CSV.read(datadir("ibm_credit_card/sd254_cards.csv"), DataFrame)
users = CSV.read(datadir("ibm_credit_card/sd254_users.csv"), DataFrame)
_transactions = CSV.read(datadir("ibm_credit_card/credit_card_transactions-ibm_v2.csv"), DataFrame)

users[!, "User"] = 0:nrow(users)-1
result_df = leftjoin(_transactions, users, on="User");
transactions = leftjoin(result_df, cards, on = ["User", "Card"] .=> ["User", "CARD INDEX"]);

In [4]:
# Create a datetime column

transactions[!, :datetime] = DateTime.(
    transactions[!, :Year],
    transactions[!, :Month],
    transactions[!, :Day],
    hour.(transactions[!, :Time]),
    minute.(transactions[!, :Time]),
)

function convert_time_features(df::DataFrame, datetime_column_name::Symbol=:datetime)
    # Extract the date column
    dates = df[!, datetime_column_name]

    # Calculate the angles for each time feature
    day_of_week_angle = 2 * π * Dates.dayofweek.(dates) / 7
    day_of_month_angle = 2 * π * (Dates.day.(dates) .- 1) ./ (Dates.daysinmonth.(dates) .- 1)
    month_angle = 2 * π * (Dates.month.(dates) .- 1) / 12

    # Transform the angles into continuous features using sine and cosine functions
    day_of_week = hcat(sin.(day_of_week_angle), cos.(day_of_week_angle))
    day_of_month = hcat(sin.(day_of_month_angle), cos.(day_of_month_angle))
    month = hcat(sin.(month_angle), cos.(month_angle))

    # Combine the continuous features into a single array
    time_features = hcat(day_of_week, day_of_month, month)

    return time_features
end

# Convert time features and add to the original dataframe
t = round.(convert_time_features(transactions), digits=3)
time_features = rename!(DataFrame(t, :auto), [:sin_weekday, :cos_weekday, :sin_monthday, :cos_monthday, :sin_month, :cos_month])

# Create new dataframe with desired features
df = hcat(transactions, time_features);

In [5]:
first(df, 5)

Row,User,Card,Year,Month,Day,Time,Amount,Use Chip,Merchant Name,Merchant City,Merchant State,Zip,MCC,Errors?,Is Fraud?,Person,Current Age,Retirement Age,Birth Year,Birth Month,Gender,Address,Apartment,City,State,Zipcode,Latitude,Longitude,Per Capita Income - Zipcode,Yearly Income - Person,Total Debt,FICO Score,Num Credit Cards,Card Brand,Card Type,Card Number,Expires,CVV,Has Chip,Cards Issued,Credit Limit,Acct Open Date,Year PIN last Changed,Card on Dark Web,datetime,sin_weekday,cos_weekday,sin_monthday,cos_monthday,sin_month,cos_month
,Int64,Int64,Int64,Int64,Int64,Time,String15,String31,Int64,String31,String?,Float64?,Int64,String?,String3,String31?,Int64?,Int64?,Int64?,Int64?,String7?,String?,Int64?,String31?,String3?,Int64?,Float64?,Float64?,String7?,String7?,String7?,Int64?,Int64?,String15?,String15?,Int64?,String7?,Int64?,String3?,Int64?,String7?,String7?,Int64?,String3?,DateTime,Float64,Float64,Float64,Float64,Float64,Float64
1,0,0,2002,9,1,06:21:00,$134.09,Swipe Transaction,3527213246127876953,La Verne,CA,91750.0,5300,missing,No,Hazel Robinson,53,66,1966,11,Female,462 Rose Lane,missing,La Verne,CA,91750,34.15,-117.76,$29278,$59696,$127613,787,5,Visa,Debit,4344676511950444,12/2022,623,YES,2,$24295,09/2002,2008,No,2002-09-01T06:21:00,-0.0,1.0,0.0,1.0,-0.866,-0.5
2,0,0,2002,9,1,06:42:00,$38.48,Swipe Transaction,-727612092139916043,Monterey Park,CA,91754.0,5411,missing,No,Hazel Robinson,53,66,1966,11,Female,462 Rose Lane,missing,La Verne,CA,91750,34.15,-117.76,$29278,$59696,$127613,787,5,Visa,Debit,4344676511950444,12/2022,623,YES,2,$24295,09/2002,2008,No,2002-09-01T06:42:00,-0.0,1.0,0.0,1.0,-0.866,-0.5
3,0,0,2002,9,2,06:22:00,$120.34,Swipe Transaction,-727612092139916043,Monterey Park,CA,91754.0,5411,missing,No,Hazel Robinson,53,66,1966,11,Female,462 Rose Lane,missing,La Verne,CA,91750,34.15,-117.76,$29278,$59696,$127613,787,5,Visa,Debit,4344676511950444,12/2022,623,YES,2,$24295,09/2002,2008,No,2002-09-02T06:22:00,0.782,0.623,0.215,0.977,-0.866,-0.5
4,0,0,2002,9,2,17:45:00,$128.95,Swipe Transaction,3414527459579106770,Monterey Park,CA,91754.0,5651,missing,No,Hazel Robinson,53,66,1966,11,Female,462 Rose Lane,missing,La Verne,CA,91750,34.15,-117.76,$29278,$59696,$127613,787,5,Visa,Debit,4344676511950444,12/2022,623,YES,2,$24295,09/2002,2008,No,2002-09-02T17:45:00,0.782,0.623,0.215,0.977,-0.866,-0.5
5,0,0,2002,9,3,06:23:00,$104.71,Swipe Transaction,5817218446178736267,La Verne,CA,91750.0,5912,missing,No,Hazel Robinson,53,66,1966,11,Female,462 Rose Lane,missing,La Verne,CA,91750,34.15,-117.76,$29278,$59696,$127613,787,5,Visa,Debit,4344676511950444,12/2022,623,YES,2,$24295,09/2002,2008,No,2002-09-03T06:23:00,0.975,-0.223,0.42,0.908,-0.866,-0.5


In [7]:
# Parse the amounts into normal floating values
# logarithmize here

df[!, :signed_amount] = map(x -> parse(Float64, x[2:end]), df[!, :Amount])

vals = repeat(["outbound"], nrow(df))
df[!, :direction] = vals
df[df[!, :signed_amount] .< 0, :direction] .= "inbound"

df[!, :amount] = log.(1 .+ abs.(df[!, :signed_amount]))

df[!, :yearly_income] = log.(1 .+ map(x -> parse(Float64, x[2:end]), df[!, "Yearly Income - Person"]))
df[!, :per_capita_income] = log.(1 .+ map(x -> parse(Float64, x[2:end]), df[!, "Per Capita Income - Zipcode"]))
df[!, :credit_limit] = log.(1 .+ map(x -> parse(Float64, x[2:end]), df[!, "Credit Limit"]))
df[!, :total_debt] = log.(1 .+ map(x -> parse(Float64, x[2:end]), df[!, "Total Debt"]));

In [8]:
# Create dictionary from the dataframe values

d = Dict(x => y for (x, y) in zip(names(df), values(df[1, :])))

parse2dict(row::DataFrameRow, keys) = Dict(x => y for (x, y) in zip(keys, values(row)))
parse2dict(row::DataFrameRow) = Dict(x => y for (x, y) in zip(names(row), values(row)))
parse2dict(df::DataFrame, row::Int, keys) = Dict(x => y for (x, y) in zip(keys, values(df[row, :])))
parse2dict(df::DataFrame, row::Int) = Dict(x => y for (x, y) in zip(names(df), values(df[row, :])))

parse2dict (generic function with 4 methods)

In [43]:
# d = Dict(
#     :scalar => 12.1,
#     :string => "string",
#     :categorical => "Yes"
# )

Dict{Symbol, Any} with 3 entries:
  :string      => "string"
  :categorical => "Yes"
  :scalar      => 12.1

In [44]:
ex = ExtractDict(
    Dict(
        :scalar => ExtractScalar(),
        :string => ExtractString(),
        :categorical => ExtractCategorical(["Yes", "No"]),
    )
)
printtree(ex)

Dict


  ├─────── string: String
  ├── categorical: Categorical d = 3
  ╰─────── scalar: Float32


In [45]:
pn = ex(d)

ProductNode  # 1 obs, 56 bytes
  ├─────── string: ArrayNode(2053×1 NGramMatrix with Union{Missing, Int64} ele ⋯
  ├── categorical: ArrayNode(3×1 MaybeHotMatrix with Union{Missing, Bool} elem ⋯
  ╰─────── scalar: ArrayNode(1×1 Array with Union{Missing, Float32} elements)  ⋯

In [46]:
m = reflectinmodel(pn)

ProductModel ↦ Dense(21 => 10)  # 2 arrays, 220 params, 960 bytes
  ├─────── string: ArrayModel([postimputing]Dense(2053 => 10))  # 3 arrays, 20 ⋯
  ├── categorical: ArrayModel([postimputing]Dense(3 => 10))  # 3 arrays, 50 pa ⋯
  ╰─────── scalar: ArrayModel([preimputing]Dense(1 => 1))  # 3 arrays, 3 param ⋯

In [48]:
Flux.setup(Flux.Optimisers.Adam(), m)

ErrorException: model must be fully mutable for `train!` to work, got `x::PostImputingMatrix{Float32, Matrix{Float32}, Vector{Float32}}`.
If `x .+= dx` is in fact ok, define `Optimisers.maywrite(::PostImputingMatrix{Float32, Matrix{Float32}, Vector{Float32}}) = true`

In [9]:
parse2dict(df, 1000)

Dict{String, Any} with 58 entries:
  "Year PIN last Changed" => 2008
  "Card on Dark Web"      => String3("No")
  "City"                  => String31("La Verne")
  "Card Number"           => 4344676511950444
  "Time"                  => 16:39:00
  "Amount"                => String15("\$26.68")
  "State"                 => String3("CA")
  "cos_monthday"          => 1.0
  "Expires"               => String7("12/2022")
  "Apartment"             => missing
  "Month"                 => 10
  "Use Chip"              => String31("Swipe Transaction")
  "User"                  => 0
  "signed_amount"         => 26.68
  "Card Type"             => String15("Debit")
  "Birth Month"           => 11
  "Merchant Name"         => 838425044734233142
  "Merchant City"         => String31("Mira Loma")
  "Num Credit Cards"      => 5
  ⋮                       => ⋮

# Extractor creation

In [9]:
first(df, 1)

Row,User,Card,Year,Month,Day,Time,Amount,Use Chip,Merchant Name,Merchant City,Merchant State,Zip,MCC,Errors?,Is Fraud?,Person,Current Age,Retirement Age,Birth Year,Birth Month,Gender,Address,Apartment,City,State,Zipcode,Latitude,Longitude,Per Capita Income - Zipcode,Yearly Income - Person,Total Debt,FICO Score,Num Credit Cards,Card Brand,Card Type,Card Number,Expires,CVV,Has Chip,Cards Issued,Credit Limit,Acct Open Date,Year PIN last Changed,Card on Dark Web,datetime,sin_weekday,cos_weekday,sin_monthday,cos_monthday,sin_month,cos_month,signed_amount,direction,amount,yearly_income,per_capita_income,credit_limit,total_debt
,Int64,Int64,Int64,Int64,Int64,Time,String15,String31,Int64,String31,String?,Float64?,Int64,String?,String3,String31?,Int64?,Int64?,Int64?,Int64?,String7?,String?,Int64?,String31?,String3?,Int64?,Float64?,Float64?,String7?,String7?,String7?,Int64?,Int64?,String15?,String15?,Int64?,String7?,Int64?,String3?,Int64?,String7?,String7?,Int64?,String3?,DateTime,Float64,Float64,Float64,Float64,Float64,Float64,Float64,String,Float64,Float64,Float64,Float64,Float64
1,0,0,2002,9,1,06:21:00,$134.09,Swipe Transaction,3527213246127876953,La Verne,CA,91750.0,5300,missing,No,Hazel Robinson,53,66,1966,11,Female,462 Rose Lane,missing,La Verne,CA,91750,34.15,-117.76,$29278,$59696,$127613,787,5,Visa,Debit,4344676511950444,12/2022,623,YES,2,$24295,09/2002,2008,No,2002-09-01T06:21:00,-0.0,1.0,0.0,1.0,-0.866,-0.5,134.09,outbound,4.90594,10.997,10.2846,10.0981,11.7568


In [10]:
df[!, :Apartment] = collect(Missings.replace(df[!, :Apartment], 0));

In [11]:
merchant_address = df[!, "Merchant City"] .* ", " .* df[!, "Merchant State"] .* string.(df[!, "Zip"])
customer_address = df[!, "Address"] .* ", " .* string.(df[!, "Apartment"]) .* ", " .* df[!, "City"] .* ", " .* df[!, "State"]

df[!, "merchant_address"] = merchant_address
df[!, "customer_address"] = customer_address;

In [12]:
scalar_features = vcat(
    names(time_features),
    "FICO Score", "Num Credit Cards",
    "amount", "yearly_income", "per_capita_income", "credit_limit", "total_debt",
    "Latitude", "Longitude",
    "Current Age", "Retirement Age"
)
categorical_features = ["Card Type", "Use Chip", "Card Brand", "Card Type", "Gender"]
string_features = ["merchant_address", "customer_address"]

2-element Vector{String}:
 "merchant_address"
 "customer_address"

In [13]:
ex = ExtractDict(
    Dict(
        (scalar_features .=> repeat([ExtractScalar()], length(scalar_features)))...,
        map(x -> x => ExtractCategorical(unique(df[!, x])), categorical_features)...,
        map(x -> x => ExtractString(), string_features)...,
    )
)
printtree(ex)

Dict


  ├─── customer_address: String
  ├───────────── Gender: Categorical d = 3


  ├──────── cos_weekday: Float32
  ├───────────── amount: Float32
  ├─────────── Latitude: Float32
  ├───── Retirement Age: Float32
  ├────────── sin_month: Float32
  ├────────── cos_month: Float32
  ├───────── Card Brand: Categorical d = 5
  ├──────── Current Age: Float32
  ├─────── cos_monthday: Float32
  ├─────── credit_limit: Float32
  ├─── merchant_address: String
  ├─── Num Credit Cards: Float32
  ├─────────── Use Chip: Categorical d = 4
  ├────── yearly_income: Float32
  ├───────── total_debt: Float32
  ├─────── sin_monthday: Float32
  ├──────── sin_weekday: Float32
  ├────────── Card Type: Categorical d = 4
  ├───────── FICO Score: Float32
  ├── per_capita_income: Float32
  ╰────────── Longitude: Float32


In [14]:
nodes = [ex(parse2dict(df, i)) for i in 1:10]

10-element Vector{ProductNode{NamedTuple{(:credit_limit, :per_capita_income, Symbol("Card Brand"), :Latitude, :merchant_address, :amount, Symbol("Current Age"), :sin_monthday, :customer_address, :total_debt, :cos_monthday, :cos_weekday, Symbol("Num Credit Cards"), :sin_month, Symbol("FICO Score"), :Longitude, :Gender, :cos_month, Symbol("Use Chip"), :sin_weekday, :yearly_income, Symbol("Retirement Age"), Symbol("Card Type")), Tuple{ArrayNode{Matrix{Union{Missing, Float32}}, Nothing}, ArrayNode{Matrix{Union{Missing, Float32}}, Nothing}, ArrayNode{MaybeHotMatrix{Union{Missing, UInt32}, UInt32, Union{Missing, Bool}}, Nothing}, ArrayNode{Matrix{Union{Missing, Float32}}, Nothing}, ArrayNode{NGramMatrix{Union{Missing, String}, Vector{Union{Missing, String}}, Union{Missing, Int64}}, Nothing}, ArrayNode{Matrix{Union{Missing, Float32}}, Nothing}, ArrayNode{Matrix{Union{Missing, Float32}}, Nothing}, ArrayNode{Matrix{Union{Missing, Float32}}, Nothing}, ArrayNode{NGramMatrix{Union{Missing, String}

In [15]:
mill_model = reflectinmodel(nodes[1])

ProductModel ↦ Dense(77 => 10)  # 2 arrays, 780 params, 3.125 KiB
  ├─────── credit_limit: ArrayModel([preimputing]Dense(1 => 1))  # 3 arrays, 3 ⋯
  ├── per_capita_income: ArrayModel([preimputing]Dense(1 => 1))  # 3 arrays, 3 ⋯
  ├───────── Card Brand: ArrayModel([postimputing]Dense(5 => 10))  # 3 arrays, ⋯
  ├─────────── Latitude: ArrayModel([preimputing]Dense(1 => 1))  # 3 arrays, 3 ⋯
  ├─── merchant_address: ArrayModel([postimputing]Dense(2053 => 10))  # 3 arra ⋯
  ├───────────── amount: ArrayModel([preimputing]Dense(1 => 1))  # 3 arrays, 3 ⋯
  ├──────── Current Age: ArrayModel([preimputing]Dense(1 => 1))  # 3 arrays, 3 ⋯
  ├─────── sin_monthday: ArrayModel([preimputing]Dense(1 => 1))  # 3 arrays, 3 ⋯
  ┊
  ├───── Retirement Age: ArrayModel([preimputing]Dense(1 => 1))  # 3 arrays, 3 ⋯
  ╰────────── Card Type: ArrayModel([postimputing]Dense(4 => 10))  # 3 arrays, ⋯

In [16]:
mill_model.(nodes)

10-element Vector{Matrix{Float32}}:
 [-92.83612; 90.48936; … ; -56.0861; 37.19169;;]
 [-92.78943; 90.79102; … ; -55.882412; 36.918625;;]
 [-92.65889; 90.64778; … ; -55.901215; 37.0719;;]
 [-92.66753; 90.636604; … ; -55.909164; 37.087563;;]
 [-92.35085; 90.623856; … ; -55.99839; 36.862385;;]
 [-92.43224; 90.73329; … ; -55.93122; 36.815575;;]
 [-92.45906; 90.7355; … ; -56.07525; 36.704548;;]
 [-92.49335; 90.69112; … ; -56.10681; 36.766727;;]
 [-92.578285; 90.7067; … ; -56.137882; 36.57105;;]
 [-92.46035; 90.61812; … ; -56.19001; 36.54554;;]

In [17]:
mill_model(nodes[1])

10×1 Matrix{Float32}:
  -92.83612
   90.48936
  -26.306198
 -129.68152
   46.53528
  -65.43753
   31.128414
  -37.14952
  -56.0861
   37.19169

# Building a RNN on HMIL features

In [18]:
using Flux

In [19]:
data = [n for n in nodes]

10-element Vector{ProductNode{NamedTuple{(:credit_limit, :per_capita_income, Symbol("Card Brand"), :Latitude, :merchant_address, :amount, Symbol("Current Age"), :sin_monthday, :customer_address, :total_debt, :cos_monthday, :cos_weekday, Symbol("Num Credit Cards"), :sin_month, Symbol("FICO Score"), :Longitude, :Gender, :cos_month, Symbol("Use Chip"), :sin_weekday, :yearly_income, Symbol("Retirement Age"), Symbol("Card Type")), Tuple{ArrayNode{Matrix{Union{Missing, Float32}}, Nothing}, ArrayNode{Matrix{Union{Missing, Float32}}, Nothing}, ArrayNode{MaybeHotMatrix{Union{Missing, UInt32}, UInt32, Union{Missing, Bool}}, Nothing}, ArrayNode{Matrix{Union{Missing, Float32}}, Nothing}, ArrayNode{NGramMatrix{Union{Missing, String}, Vector{Union{Missing, String}}, Union{Missing, Int64}}, Nothing}, ArrayNode{Matrix{Union{Missing, Float32}}, Nothing}, ArrayNode{Matrix{Union{Missing, Float32}}, Nothing}, ArrayNode{Matrix{Union{Missing, Float32}}, Nothing}, ArrayNode{NGramMatrix{Union{Missing, String}

In [20]:
mill_model = reflectinmodel(nodes[1])
rnn_model = RNN(10, 5)
model = Chain(mill_model, rnn_model, Dense(5, 1))

Recur(
  RNNCell(10 => 5, tanh),               # 85 parameters
)         # Total: 4 trainable arrays, 85 parameters,
          # plus 1 non-trainable, 5 parameters, summarysize 572 bytes.

In [21]:
rnn_model(randn(Float32, 10))

5-element Vector{Float32}:
 -0.040493954
  0.89472073
  0.9207458
  0.8082733
 -0.9519195

Chain(
  ProductModel(
    NamedTuple(
      ArrayModel(
        [preimputing]Dense(1 => 1),     # 2 parameters
      ),
      ArrayModel(
        [preimputing]Dense(1 => 1),     # 2 parameters
      ),
      ArrayModel(
        [postimputing]Dense(5 => 10),   # 60 parameters
      ),
      ArrayModel(
        [preimputing]Dense(1 => 1),     # 2 parameters
      ),
      ArrayModel(
        [postimputing]Dense(2053 => 10),  # 20_540 parameters
      ),
      ArrayModel(
        [preimputing]Dense(1 => 1),     # 2 parameters
      ),
      ArrayModel(
        [preimputing]Dense(1 => 1),     # 2 parameters
      ),
      ArrayModel(
        [preimputing]Dense(1 => 1),     # 2 parameters
      ),
      ArrayModel(
        [postimputing]Dense(2053 => 10),  # 20_540 parameters
      ),
      ArrayModel(
        [preimputing]Dense(1 => 1),     # 2 parameters
      ),
      ArrayModel(
        [preimputing]Dense(1 => 1),     # 2 parameters
      ),
      ArrayModel(
        [preimputing]Dense

In [23]:
using StatsBase

In [24]:
pos_ix = collect(1:nrow(df))[df[!, "Is Fraud?"] .== "Yes"]

29757-element Vector{Int64}:
     4100
     4101
     4102
     4103
     4104
     4105
     4106
     4107
     4108
     4109
        ⋮
 24372754
 24372755
 24372758
 24372761
 24375669
 24375671
 24375672
 24375673
 24375674

In [25]:
# Get a sample of data

ix = sample(1:1_000_000, 100)
ix = vcat(ix, sample(pos_ix, 100))
data = [ex(parse2dict(df, i)) for i in ix]
labels = df[ix, "Is Fraud?"] .== "Yes"
sum(labels)

100

In [26]:
yhat = [model(d) for d in data];

In [27]:
using Flux: binarycrossentropy, logitbinarycrossentropy

In [28]:
logitbinarycrossentropy.(yhat, labels)

200-element Vector{Float32}:
 1.3946197
 1.3716677
 1.2449713
 1.39462
 1.3135533
 1.3946198
 1.22293
 1.3945265
 1.3607221
 1.3256472
 ⋮
 0.35206398
 0.28492266
 0.32987806
 0.28492218
 0.35189986
 0.28546175
 0.28492218
 0.3187511
 0.28497922

In [29]:
function loss(x, y)
    Flux.reset!(rnn_model)
    yhat = [model(xi) for xi in x]
    return mean(logitbinarycrossentropy.(yhat, y))
end
loss(x) = loss(x...)

loss (generic function with 2 methods)

In [30]:
loss(data, labels)

0.82946515f0

In [31]:
opt = ADAM()

Adam(0.001, (0.9, 0.999), 1.0e-8, IdDict{Any, Any}())

In [32]:
ps = Flux.params(model)

Params([PreImputingMatrix(W = Float32[1.0;;], ψ = Float32[0.0]), Float32[0.0], PreImputingMatrix(W = Float32[1.0;;], ψ = Float32[0.0]), Float32[0.0], PostImputingMatrix(W = Float32[-0.198444 0.0839636 -0.255636 -0.441605 -0.399543; -0.283381 0.345318 0.538266 -0.584664 -0.232448; 0.306403 -0.630693 -0.362065 -0.315797 0.0611841; 0.0633426 -0.498817 0.397576 -0.570386 -0.606191; 0.628369 -0.559433 0.567582 -0.538993 -0.397489; 0.111039 0.0820731 0.350265 0.123328 -0.357201; 0.208725 -0.249683 -0.146591 0.00565934 0.442089; 0.538994 0.30974 -0.631138 0.287008 0.569088; -0.150554 -0.234627 0.383624 -0.45762 -0.0180192; -0.117304 0.229814 0.288504 0.158111 -0.316224], ψ = Float32[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]), Float32[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], PreImputingMatrix(W = Float32[1.0;;], ψ = Float32[0.0]), Float32[0.0], PostImputingMatrix(W = Float32[-0.0368723 -0.0526919 0.0515352 -0.0223222 -0.0259373 0.0467586 -0.0334485 0.0124982 0.0170948 -0.0173

In [34]:
using Flux.Optimisers

In [35]:
# Optimisers.maywrite(::PreImputingMatrix{Float32, Matrix{Float32}, Vector{Float32}}) = true
# Optimisers.maywrite(::PostImputingMatrix{Float32, Matrix{Float32}, Vector{Float32}}) = true

In [33]:
X = [data[i:i+10] for i in 1:10:190]

19-element Vector{Vector{ProductNode{NamedTuple{(:credit_limit, :per_capita_income, Symbol("Card Brand"), :Latitude, :merchant_address, :amount, Symbol("Current Age"), :sin_monthday, :customer_address, :total_debt, :cos_monthday, :cos_weekday, Symbol("Num Credit Cards"), :sin_month, Symbol("FICO Score"), :Longitude, :Gender, :cos_month, Symbol("Use Chip"), :sin_weekday, :yearly_income, Symbol("Retirement Age"), Symbol("Card Type")), Tuple{ArrayNode{Matrix{Union{Missing, Float32}}, Nothing}, ArrayNode{Matrix{Union{Missing, Float32}}, Nothing}, ArrayNode{MaybeHotMatrix{Union{Missing, UInt32}, UInt32, Union{Missing, Bool}}, Nothing}, ArrayNode{Matrix{Union{Missing, Float32}}, Nothing}, ArrayNode{NGramMatrix{Union{Missing, String}, Vector{Union{Missing, String}}, Union{Missing, Int64}}, Nothing}, ArrayNode{Matrix{Union{Missing, Float32}}, Nothing}, ArrayNode{Matrix{Union{Missing, Float32}}, Nothing}, ArrayNode{Matrix{Union{Missing, Float32}}, Nothing}, ArrayNode{NGramMatrix{Union{Missing, 

In [34]:
Y = [labels[i:i+10] for i in 1:10:190]

19-element Vector{BitVector}:
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]
 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]

In [35]:
loss(X[1], Y[1])

1.336383f0

In [36]:
Flux.reset!(rnn_model)

5×1 Matrix{Float32}:
 0.0
 0.0
 0.0
 0.0
 0.0

In [38]:
model(X[1])

1×11 Matrix{Float32}:
 1.1097  1.07906  0.894826  1.1097  1.0004  …  1.06432  0.988072  0.909042

In [41]:
data

200-element Vector{ProductNode{NamedTuple{(:credit_limit, :per_capita_income, Symbol("Card Brand"), :Latitude, :merchant_address, :amount, Symbol("Current Age"), :sin_monthday, :customer_address, :total_debt, :cos_monthday, :cos_weekday, Symbol("Num Credit Cards"), :sin_month, Symbol("FICO Score"), :Longitude, :Gender, :cos_month, Symbol("Use Chip"), :sin_weekday, :yearly_income, Symbol("Retirement Age"), Symbol("Card Type")), Tuple{ArrayNode{Matrix{Union{Missing, Float32}}, Nothing}, ArrayNode{Matrix{Union{Missing, Float32}}, Nothing}, ArrayNode{MaybeHotMatrix{Union{Missing, UInt32}, UInt32, Union{Missing, Bool}}, Nothing}, ArrayNode{Matrix{Union{Missing, Float32}}, Nothing}, ArrayNode{NGramMatrix{Union{Missing, String}, Vector{Union{Missing, String}}, Union{Missing, Int64}}, Nothing}, ArrayNode{Matrix{Union{Missing, Float32}}, Nothing}, ArrayNode{Matrix{Union{Missing, Float32}}, Nothing}, ArrayNode{Matrix{Union{Missing, Float32}}, Nothing}, ArrayNode{NGramMatrix{Union{Missing, String

In [44]:
X[1][1]

ProductNode  # 1 obs, 264 bytes
  ├─────── credit_limit: ArrayNode(1×1 Array with Union{Missing, Float32} elem ⋯
  ├── per_capita_income: ArrayNode(1×1 Array with Union{Missing, Float32} elem ⋯
  ├───────── Card Brand: ArrayNode(5×1 MaybeHotMatrix with Union{Missing, Bool ⋯
  ├─────────── Latitude: ArrayNode(1×1 Array with Union{Missing, Float32} elem ⋯
  ├─── merchant_address: ArrayNode(2053×1 NGramMatrix with Union{Missing, Int6 ⋯
  ├───────────── amount: ArrayNode(1×1 Array with Union{Missing, Float32} elem ⋯
  ├──────── Current Age: ArrayNode(1×1 Array with Union{Missing, Float32} elem ⋯
  ├─────── sin_monthday: ArrayNode(1×1 Array with Union{Missing, Float32} elem ⋯
  ┊
  ├───── Retirement Age: ArrayNode(1×1 Array with Union{Missing, Float32} elem ⋯
  ╰────────── Card Type: ArrayNode(4×1 MaybeHotMatrix with Union{Missing, Bool ⋯

In [45]:
logitbinarycrossentropy(X[1][1], Y[1][1])

MethodError: MethodError: no method matching length(::ProductNode{NamedTuple{(:credit_limit, :per_capita_income, Symbol("Card Brand"), :Latitude, :merchant_address, :amount, Symbol("Current Age"), :sin_monthday, :customer_address, :total_debt, :cos_monthday, :cos_weekday, Symbol("Num Credit Cards"), :sin_month, Symbol("FICO Score"), :Longitude, :Gender, :cos_month, Symbol("Use Chip"), :sin_weekday, :yearly_income, Symbol("Retirement Age"), Symbol("Card Type")), Tuple{ArrayNode{Matrix{Union{Missing, Float32}}, Nothing}, ArrayNode{Matrix{Union{Missing, Float32}}, Nothing}, ArrayNode{MaybeHotMatrix{Union{Missing, UInt32}, UInt32, Union{Missing, Bool}}, Nothing}, ArrayNode{Matrix{Union{Missing, Float32}}, Nothing}, ArrayNode{NGramMatrix{Union{Missing, String}, Vector{Union{Missing, String}}, Union{Missing, Int64}}, Nothing}, ArrayNode{Matrix{Union{Missing, Float32}}, Nothing}, ArrayNode{Matrix{Union{Missing, Float32}}, Nothing}, ArrayNode{Matrix{Union{Missing, Float32}}, Nothing}, ArrayNod

In [50]:
X[1]

11-element Vector{ProductNode{NamedTuple{(:credit_limit, :per_capita_income, Symbol("Card Brand"), :Latitude, :merchant_address, :amount, Symbol("Current Age"), :sin_monthday, :customer_address, :total_debt, :cos_monthday, :cos_weekday, Symbol("Num Credit Cards"), :sin_month, Symbol("FICO Score"), :Longitude, :Gender, :cos_month, Symbol("Use Chip"), :sin_weekday, :yearly_income, Symbol("Retirement Age"), Symbol("Card Type")), Tuple{ArrayNode{Matrix{Union{Missing, Float32}}, Nothing}, ArrayNode{Matrix{Union{Missing, Float32}}, Nothing}, ArrayNode{MaybeHotMatrix{Union{Missing, UInt32}, UInt32, Union{Missing, Bool}}, Nothing}, ArrayNode{Matrix{Union{Missing, Float32}}, Nothing}, ArrayNode{NGramMatrix{Union{Missing, String}, Vector{Union{Missing, String}}, Union{Missing, Int64}}, Nothing}, ArrayNode{Matrix{Union{Missing, Float32}}, Nothing}, ArrayNode{Matrix{Union{Missing, Float32}}, Nothing}, ArrayNode{Matrix{Union{Missing, Float32}}, Nothing}, ArrayNode{NGramMatrix{Union{Missing, String}

In [52]:
Flux.reset!(rnn_model)

5×1 Matrix{Float32}:
 0.0
 0.0
 0.0
 0.0
 0.0

In [53]:
mean(logitbinarycrossentropy(model.(X[1]), Y[1]))

1.336383f0

In [55]:
Flux.gradient(m -> mean(loss((X[1]), X[1])), model)

MethodError: MethodError: no method matching length(::ProductNode{NamedTuple{(:credit_limit, :per_capita_income, Symbol("Card Brand"), :Latitude, :merchant_address, :amount, Symbol("Current Age"), :sin_monthday, :customer_address, :total_debt, :cos_monthday, :cos_weekday, Symbol("Num Credit Cards"), :sin_month, Symbol("FICO Score"), :Longitude, :Gender, :cos_month, Symbol("Use Chip"), :sin_weekday, :yearly_income, Symbol("Retirement Age"), Symbol("Card Type")), Tuple{ArrayNode{Matrix{Union{Missing, Float32}}, Nothing}, ArrayNode{Matrix{Union{Missing, Float32}}, Nothing}, ArrayNode{MaybeHotMatrix{Union{Missing, UInt32}, UInt32, Union{Missing, Bool}}, Nothing}, ArrayNode{Matrix{Union{Missing, Float32}}, Nothing}, ArrayNode{NGramMatrix{Union{Missing, String}, Vector{Union{Missing, String}}, Union{Missing, Int64}}, Nothing}, ArrayNode{Matrix{Union{Missing, Float32}}, Nothing}, ArrayNode{Matrix{Union{Missing, Float32}}, Nothing}, ArrayNode{Matrix{Union{Missing, Float32}}, Nothing}, ArrayNod

In [54]:
Flux.gradient(m -> mean(logitbinarycrossentropy(m.(X[1]), X[1])), model)

MethodError: MethodError: no method matching -(::Int64, ::ProductNode{NamedTuple{(:credit_limit, :per_capita_income, Symbol("Card Brand"), :Latitude, :merchant_address, :amount, Symbol("Current Age"), :sin_monthday, :customer_address, :total_debt, :cos_monthday, :cos_weekday, Symbol("Num Credit Cards"), :sin_month, Symbol("FICO Score"), :Longitude, :Gender, :cos_month, Symbol("Use Chip"), :sin_weekday, :yearly_income, Symbol("Retirement Age"), Symbol("Card Type")), Tuple{ArrayNode{Matrix{Union{Missing, Float32}}, Nothing}, ArrayNode{Matrix{Union{Missing, Float32}}, Nothing}, ArrayNode{MaybeHotMatrix{Union{Missing, UInt32}, UInt32, Union{Missing, Bool}}, Nothing}, ArrayNode{Matrix{Union{Missing, Float32}}, Nothing}, ArrayNode{NGramMatrix{Union{Missing, String}, Vector{Union{Missing, String}}, Union{Missing, Int64}}, Nothing}, ArrayNode{Matrix{Union{Missing, Float32}}, Nothing}, ArrayNode{Matrix{Union{Missing, Float32}}, Nothing}, ArrayNode{Matrix{Union{Missing, Float32}}, Nothing}, Arra

In [46]:
grads = Flux.gradient(m -> mean(logitbinarycrossentropy(m(X[1]), X[1])), model)

DimensionMismatch: DimensionMismatch: loss function expects size(ŷ) = (1, 11) to match size(y) = (11,)

In [42]:
Flux.setup(Adam(), mill_model)

ErrorException: model must be fully mutable for `train!` to work, got `x::PreImputingMatrix{Float32, Matrix{Float32}, Vector{Float32}}`.
If `x .+= dx` is in fact ok, define `Optimisers.maywrite(::PreImputingMatrix{Float32, Matrix{Float32}, Vector{Float32}}) = true`

In [41]:
opt_state = Flux.setup(Adam(), model)

ErrorException: model must be fully mutable for `train!` to work, got `x::PreImputingMatrix{Float32, Matrix{Float32}, Vector{Float32}}`.
If `x .+= dx` is in fact ok, define `Optimisers.maywrite(::PreImputingMatrix{Float32, Matrix{Float32}, Vector{Float32}}) = true`

In [80]:
for epoch in 1:100
  for data in zip(X, Y)
    # Unpack this element (for supervised training):
    input, label = data

    # Reset the RNN state
    Flux.reset!(rnn_model)

    # Calculate the gradient of the objective
    # with respect to the parameters within the model:
    grads = Flux.gradient(model) do m
      out = [m(x) for x in input]
      mean(logitbinarycrossentropy.(out, label))
    end

    # Update the parameters so as to reduce the objective,
    # according the chosen optimisation rule:
    Flux.update!(opt, ps, grads[1])
  end
end

ErrorException: can't mix implicit Params with explicit gradients!
* For the implicit style, this needs `update(::AbstractOptimiser, ::Params, ::Grads)` with implicit gradient.
* For the explicit style, `update(state, model, grad)` needs the model itself, and `state = Flux.setup(opt, model)`.


In [125]:
loss(X[1], Y[1])

1.8381721f0

In [86]:
gradient(loss, X[1], Y[1])

(NamedTuple{(:data, :metadata), Tuple{NamedTuple{(:credit_limit, :per_capita_income, Symbol("Card Brand"), :Latitude, :merchant_address, :amount, Symbol("Current Age"), :sin_monthday, :customer_address, :total_debt, :cos_monthday, :cos_weekday, Symbol("Num Credit Cards"), :sin_month, Symbol("FICO Score"), :Longitude, :Gender, :cos_month, Symbol("Use Chip"), :sin_weekday, :yearly_income, Symbol("Retirement Age"), Symbol("Card Type")), Tuple{NamedTuple{(:data, :metadata), Tuple{Matrix{Float32}, Nothing}}, NamedTuple{(:data, :metadata), Tuple{Matrix{Float32}, Nothing}}, Nothing, NamedTuple{(:data, :metadata), Tuple{Matrix{Float32}, Nothing}}, Nothing, NamedTuple{(:data, :metadata), Tuple{Matrix{Float32}, Nothing}}, NamedTuple{(:data, :metadata), Tuple{Matrix{Float32}, Nothing}}, NamedTuple{(:data, :metadata), Tuple{Matrix{Float32}, Nothing}}, Nothing, NamedTuple{(:data, :metadata), Tuple{Matrix{Float32}, Nothing}}, NamedTuple{(:data, :metadata), Tuple{Matrix{Float32}, Nothing}}, NamedTupl

In [84]:
Flux.train!(loss, model, (X, Y), opt)

MethodError: MethodError: no method matching (::ProductModel{NamedTuple{(:credit_limit, :per_capita_income, Symbol("Card Brand"), :Latitude, :merchant_address, :amount, Symbol("Current Age"), :sin_monthday, :customer_address, :total_debt, :cos_monthday, :cos_weekday, Symbol("Num Credit Cards"), :sin_month, Symbol("FICO Score"), :Longitude, :Gender, :cos_month, Symbol("Use Chip"), :sin_weekday, :yearly_income, Symbol("Retirement Age"), Symbol("Card Type")), Tuple{ArrayModel{Dense{typeof(identity), PreImputingMatrix{Float32, Matrix{Float32}, Vector{Float32}}, Vector{Float32}}}, ArrayModel{Dense{typeof(identity), PreImputingMatrix{Float32, Matrix{Float32}, Vector{Float32}}, Vector{Float32}}}, ArrayModel{Dense{typeof(identity), PostImputingMatrix{Float32, Matrix{Float32}, Vector{Float32}}, Vector{Float32}}}, ArrayModel{Dense{typeof(identity), PreImputingMatrix{Float32, Matrix{Float32}, Vector{Float32}}, Vector{Float32}}}, ArrayModel{Dense{typeof(identity), PostImputingMatrix{Float32, Matri

In [57]:
using Flux.Functors

In [59]:
using Flux, Mill, JsonGrinder#, Functors

@functor PostImputingMatrix
@functor PreImputingMatrix
# Flux.Optimisers.isnumeric(::PostImputingMatrix) = false
# Flux.Optimisers.isnumeric(::PreImputingMatrix) = false

In [81]:
mill_model = reflectinmodel(nodes[1])
rnn_model = RNN(10, 5)
model = Chain(mill_model, rnn_model, Dense(5, 1));

In [82]:
# Try to setup the optimizer

opt_state = Flux.setup(Adam(), model)
# gs = gradient(m -> sum(m(nodes)), model)[1]
# Flux.Optimisers.update!(opt_state, model, gs)

(layers = ((ms = (credit_limit = (m = (weight = (W = Leaf(Adam{Float64}(0.001, (0.9, 0.999), 1.0e-8), (Float32[0.0;;], Float32[0.0;;], (0.9, 0.999))), ψ = Leaf(Adam{Float64}(0.001, (0.9, 0.999), 1.0e-8), (Float32[0.0], Float32[0.0], (0.9, 0.999)))), bias = Leaf(Adam{Float64}(0.001, (0.9, 0.999), 1.0e-8), (Float32[0.0], Float32[0.0], (0.9, 0.999))), σ = ()),), per_capita_income = (m = (weight = (W = Leaf(Adam{Float64}(0.001, (0.9, 0.999), 1.0e-8), (Float32[0.0;;], Float32[0.0;;], (0.9, 0.999))), ψ = Leaf(Adam{Float64}(0.001, (0.9, 0.999), 1.0e-8), (Float32[0.0], Float32[0.0], (0.9, 0.999)))), bias = Leaf(Adam{Float64}(0.001, (0.9, 0.999), 1.0e-8), (Float32[0.0], Float32[0.0], (0.9, 0.999))), σ = ()),), var"Card Brand" = (m = (weight = (W = Leaf(Adam{Float64}(0.001, (0.9, 0.999), 1.0e-8), (Float32[0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0], Float32[0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0], (0.9, 0.999))), ψ = Lea

In [83]:
for epoch in 1:100
    for data in zip(X, Y)
        # Unpack this element (for supervised training):
        input, label = data

        # Reset the RNN state
        Flux.reset!(rnn_model)

        # Calculate the gradient and update the states
        gs = gradient(m -> mean(logitbinarycrossentropy.(m(input), label)), model)[1]
        Flux.Optimisers.update!(opt_state, model, gs)
    end
end

In [84]:
Flux.reset!(rnn_model)
model(nodes)

1×10 Matrix{Float32}:
 -0.0857528  -0.0857528  -0.0857528  …  -0.0857528  -0.0857528  -0.0857528

In [78]:
Flux.reset!(rnn_model)
model(nodes)

1×10 Matrix{Float32}:
 -0.0830396  -0.0830396  -0.0830396  …  -0.0830396  -0.0830396  -0.0830396